# Import libraries

In [ ]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

# check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Build the functions

In [ ]:
def tokenise(sentence):
    """
    split sentence into an array of tokens
    """
    return nltk.word_tokenize(sentence)

def lower_stem(word):
    """
    stemming = find the root form of the word
    words = ["organize", "organizes", "organizing"]
          -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())

def bag_of_words(tokenised_sentence, words):
    """
    return bag of words array: 1 indicates each known word that exists in the sentence, 0 otherwise
    tokened_sentence = ['may', 'I', 'have', 'a', 'look']
    words = ["hi", "have", "how", "may", "you", "bye", "a", "wonder"]
    bag   = [ 0,      1,     0,     1,     0,     0,    1,      0   ]   
    """
    # stem each word
    sentence_words = [lower_stem(word) for word in tokenised_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
    return bag

# Build the model

In [ ]:
'''
build neural network:
  - three linear layers
  - activation layer: ReLU
  - dropout ratio: 0.15
'''
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear_layer_1 = nn.Linear(input_size, hidden_size) 
        self.linear_layer_2 = nn.Linear(hidden_size, hidden_size) 
        self.linear_layer_3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.15)
    
    def forward(self, x):
        out = self.linear_layer_1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.linear_layer_2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.linear_layer_3(out)
        return out

# Load and build the dataset

In [ ]:
intents_path = "/content/drive/MyDrive/UCD_Programmes/Summer/ACM40960_Projects_in_Maths_Modelling/My_Project/dataset/intents.json"
with open(intents_path, 'r') as f:
    intents = json.load(f)

In [ ]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
  tag = intent['tag']
  # add to tag list
  tags.append(tag)
  for pattern in intent['patterns']:
    # tokenize each word in the sentence
    w = tokenise(pattern)
    # add to our words list
    all_words.extend(w)
    # add to xy pair
    xy.append((w, tag))

In [ ]:
# lower, stem and eliminate punctuation words 
all_words = [lower_stem(w) for w in all_words if w.isalnum()]

# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

In [ ]:
# create training data
x_train = []
y_train = []
for (tokenised_sentence, tag) in xy:
  # x: bag of words for each pattern_sentence
  bag = bag_of_words(tokenised_sentence, all_words)
  x_train.append(bag)
  # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
  label = tags.index(tag)
  y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(x_train)
    self.x_data = x_train
    self.y_data = y_train

  # support indexing such that dataset[i] can be used to get i-th sample
  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  # we can call len(dataset) to return the size
  def __len__(self):
    return self.n_samples

# Model training

In [ ]:
# set hyper-parameters 
epoches = 200
batch_size = 5
learning_rate = 0.001
input_size = x_train.shape[1]
hidden_size = 64
output_size = len(tags)
print(input_size, output_size)

# build the training dataset
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

# initialise the model
model = NN(input_size, hidden_size, output_size).to(device)

# define loss function
criterion = nn.CrossEntropyLoss()

# define optimisation function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# start training the model
for epoch in range(epoches):
  for (words, labels) in train_loader:

    # initialise the gradient cache
    optimizer.zero_grad()

    # run in the device (either GPU or CPU)
    words = words.to(device)
    labels = labels.to(dtype=torch.long).to(device)
    
    # Forward pass
    outputs = model(words)

    # calculate the loss
    loss = criterion(outputs, labels)
    
    # backward-prapagation
    loss.backward()

    # update the parameters
    optimizer.step()
      
  if (epoch+1) % 20 == 0:
    print (f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.3f}')

print(f'final loss: {loss.item():.3f}')

In [ ]:
model.eval()
with torch.no_grad():
  print("Let's chat! (type 'quit' to exit)")

  exit_list = ['exit', 'see you later', 'bye', 'quit', 'breat', 'q']
  while True:
    sentence = input("You: ")
    if sentence.lower() in exit_list:
      print("Bot: See you!")
      break

    sentence = tokenise(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.6:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"Bot: {random.choice(intent['responses'])}")
    else:
        print(f"Bot: My apologies, I don't get it...")